### ztf_sim_introduction

This notebook illustrates basic use of the `ztf_sim` modules.

In [1]:
# hack to get the path right
import sys
sys.path.append('..')

In [2]:
import ztf_sim
from astropy.time import Time

/opt/local/Library/Frameworks/Python.framework/Versions/2.7/lib/python2.7/site-packages/IPython/kernel/__init__.py:13: ShimWarning: The `IPython.kernel` package has been deprecated. You should import from ipykernel or jupyter_client instead.
  "You should import from ipykernel or jupyter_client instead.", ShimWarning)


First we'll generate a test field grid.  You only need to do this the first time you run the simulator.

In [3]:
ztf_sim.fields.generate_test_field_grid()

Let's load the Fields object with the default field grid:

In [4]:
f = ztf_sim.fields.Fields()

The raw fieldid and coordinates are stored as a pandas Dataframe in the `.fields` attribute:

In [5]:
f.fields

,dec,ra
fieldid,,
0,90,0.000000
1,83,0.000000
2,83,36.000000
3,83,72.000000
4,83,108.000000
5,83,144.000000
6,83,180.000000
7,83,216.000000
8,83,252.000000


Now let's calcuate their altitude and azimuth at a specific time using the astropy.time.Time object:

In [6]:
f.alt_az(Time.now())

If you need enough precision such that this matters (~<10 arcsec), you can
download the latest IERS predictions by running:

    >>> from astropy.utils.data import download_file
    >>> from astropy.utils import iers
    >>> iers.IERS.iers_table = iers.IERS_A.open(download_file(iers.IERS_A_URL, cache=True))

 [astropy.coordinates.builtin_frames.utils]
If you need enough precision such that this matters (~<10 arcsec), you can
download the latest IERS predictions by running:

    >>> from astropy.utils.data import download_file
    >>> from astropy.utils import iers
    >>> iers.IERS.iers_table = iers.IERS_A.open(download_file(iers.IERS_A_URL, cache=True))


If you need enough precision such that this matters (~<10 arcsec), you can
download the latest IERS predictions by running:

    >>> from astropy.utils.data import download_file
    >>> from astropy.utils import iers
    >>> iers.IERS.iers_table = iers.IERS_A.open(download_file(iers.IERS_A_URL, cache=True))

 [astropy.coordinates.bui

,alt,az
fieldid,,
0,33.410919,359.918651
1,28.478990,5.714484
2,26.525504,1.374548
3,27.034632,356.573113
4,29.841659,352.843438
5,34.012279,351.532641
6,38.041672,353.447164
7,40.276438,358.211092
8,39.683303,3.791620


Demonstrate accessing fields by `fieldid`:

In [7]:
w = f.fields['dec'] < 0.
f.fields[w].loc[853]

dec   -90
ra      0
Name: 853, dtype: float64

Calculating the overhead time (max of ha, dec, dome slews and readout time):

In [8]:
f.overhead_time(853,Time.now())

fieldid
0      157.000000
1      151.166667
2      151.166667
3      151.166667
4      151.166667
5      151.166667
6      151.166667
7      151.166667
8      151.166667
9      151.166667
10     151.166667
11     145.333333
12     145.333333
13     145.333333
14     145.333333
15     145.333333
16     145.333333
17     145.333333
18     145.333333
19     145.333333
20     145.333333
21     145.333333
22     145.333333
23     145.333333
24     145.333333
25     145.333333
26     145.333333
27     139.500000
28     139.500000
29     139.500000
          ...    
824     62.175926
825     50.925926
826     39.675926
827     28.666667
828     28.666667
829     22.833333
830     22.833333
831     38.068783
832     54.140212
833     70.211640
834     86.283069
835    102.354497
836    118.425926
837    102.354497
838     86.283069
839     70.211640
840     54.140212
841     38.068783
842     22.833333
843     17.000000
844     34.050926
845     62.175926
846     90.300926
847    118.425926
84